# Induced gap and magnetic field

# TO-DO:
* Remove the use of `delta` function in `tunnel_hoppings`
* Tune induced gap algo
* Correct for zero supercurrent because of non-zero total $A$ in SC, example from [orbitalfield](https://github.com/basnijholt/orbitalfield) project:

```python
    def peierls(val, ind, a):
        def phase(s1, s2, p):
            x, y, z = s1.pos
            A_site = [p.B_y * z - p.B_z * y, 0, p.B_x * y][ind]
            if p.A_correction:
                A_sc = [A(p, *site.pos) for site in sc_sites]
                A_site -= np.mean(A_sc, axis=0)[ind]
            A_site *= a * 1e-18 * eV / hbar
            return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz
```

In [ ]:
# 1. Standard library imports
from importlib import reload
from itertools import product
import os.path
from types import SimpleNamespace

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy
from toolz import partition_all

# 3. Internal imports
import funcs
reload(funcs)

hv.notebook_extension()
print(kwant.__version__)

# Usage

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, mu_B=funcs.constants.mu_B,
                orbital=True, c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_eff, mu_sc=10, mu_sm=10, e=funcs.constants.e, c_tunnel=7/8, V=0)

syst_pars = dict(a=10, angle=45, onsite_disorder=False,
                   L=200, phi=135, r1=50, r2=80, shape='circle',
                   with_leads=True, with_shell=True)

syst = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
%%opts Path [aspect='square']
lead = funcs.make_lead(10, 50, 70, 135, 0, True, 'circle')
ham_pars['B_x'] = 1
ham_pars['B_y'] = 0
ham_pars['orbital'] = True
ks = np.linspace(-3, 3)
Es = funcs.bands(lead, ham_pars, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

ham_pars['orbital'] = False
Es1 = funcs.bands(lead, ham_pars, ks)
p2 = hv.Path((ks, Es1))[:, -100:100]

p1 * p2

In [ ]:
(p1 * p2)[:, -1:1]

In [ ]:
%%time
gap = funcs.find_gap(lead, ham_pars)
print('The bandgap is {} meV'.format(gap))

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, mu_B=funcs.constants.mu_B,
                orbital=True, c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_e, mu_sc=5, mu_sm=5, e=funcs.constants.e, c_tunnel=7/8, V=0)

S = kwant.smatrix(syst, params=ham_pars)

In [ ]:
funcs.andreev_conductance(syst, ham_pars)

# Dask

In [ ]:
import dask
import distributed
from dask import delayed

from dask_drmaa import DRMAACluster
cluster = DRMAACluster()

from dask.distributed import Client
client = Client(cluster)
cluster.start_workers(10)

client.upload_file('combine.py')
client.upload_file('funcs.py')

# ipyparallel

In [ ]:
# Make sure this folder is in your $PYTHONPATH
import socket
if socket.gethostname() == 'hpc05':
    from ipyparallel import Client
    client = Client(profile='pbs')
else:
    try:
        # If this can be imported, it assumes you are on the TU Delft network with access to cluster
        import hpc05
        client = hpc05.Client(profile='pbs', timeout=60, extra_args='--timeout=1200')
        print("Connected to hpc05")
    except ImportError:
        from ipyparallel import Client
        client = Client()
        print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))
%px import funcs

In [ ]:
ham_pars = dict(Delta=60, mu_B=funcs.constants.mu_B,
                c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_eff, mu_sc=15, mu_sm=15, e=funcs.constants.e, c_tunnel=7/8)

syst_pars = dict(a=10, angle=45, phi=135, r1=50, r2=80, shape='circle', with_shell=True)

def gap(val, syst_pars=syst_pars, ham_pars=ham_pars):
    import funcs
    val['B_x'], val['B_y'], val['B_z'] = val.pop('B')
    ham_pars = dict(**ham_pars, **val)
    lead = funcs.make_lead(**syst_pars)
    return dict(E_gap=funcs.find_gap(lead, ham_pars), **val)


# Bs = np.linspace(0, 1, 50)
# zeros = np.zeros(Bs.shape)
# Bs = np.hstack([np.vstack([Bs, zeros, zeros]),
#                 np.vstack([zeros, Bs, zeros]),
#                 np.vstack([zeros, zeros, Bs])]).T
# Vs = np.linspace(-50, 50, 50)
# alphas = [20, 0]
# gs = [50, 0]
# orbital_bools = [True, False]


Bs = np.linspace(0, 1, 25)
zeros = np.zeros(Bs.shape)
Bs = np.vstack([Bs, zeros, zeros]).T
Vs = np.linspace(-50, 50, 25)
alphas = [20]
gs = [50]
orbital_bools = [True]

vals = funcs.named_product(B=Bs, V=Vs, g=gs, alpha=alphas, orbital=orbital_bools)
print(len(vals))

for i, chunk in enumerate(partition_all(3000, vals)):
    fname = 'test_gaps_{}.hdf'.format(i)
    if not os.path.exists(fname):
        E_gap = lview.map_async(gap, chunk)
        E_gap.wait_interactive()
        E_gap = E_gap.result()

        df = pd.DataFrame(E_gap)
        
        df = df.assign(**syst_pars).assign(**ham_pars).assign(**funcs.constants.__dict__)
        df = df.assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')